##### An example of using foresee from jupyter notebook

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import numpy as np
from io import StringIO
import importlib_resources

# import main from foresee.scripts
from foresee.scripts import main

In [ ]:
# upload sample time-series dataframe with columns(id, date_stamp, y)

test_data_light_txt = importlib_resources.files('foresee.data').joinpath('basic_time_series_data.csv').read_text()

ts_df = pd.read_csv(StringIO(test_data_light_txt))
ts_df.head()

In [ ]:
# user defind parameters

# time series values column name: required if input dataframe has more than one column

endog_colname = None

if len(ts_df.columns) > 1 and endog_colname is None:
    raise ValueError('time series column name is required!!!')

freq = 5
fcst_length = 10
# model_list = ['ewm_model']
model_list = ['ewm_model', 'fft', 'holt_winters', 'prophet', 'sarimax']

'''
avilable run types: 'best_model', 'all_best', 'all_models'
'''

run_type = 'all_models'

# if comparing models (run_type is 'best_model' or 'all_best') then holdout length is required

if run_type == 'all_models':
    holdout_length = None
else:
    holdout_length = 20


# we are working with one time series and no date-time column so time series id and date-time column name are set to None.
gbkey = None
ds_column = None


# we are fitting one time series in this example so no need to parallelize.

fit_execution_method = 'non_parallel'

In [ ]:
'''
result:  dataframe containing fitted values and future forecasts
fit_results_list:  list of dictionaries containing fitted values, forecasts, and errors (useful for debuging)
'''

result, fit_result_list = main.collect_result(
                                                    ts_df.copy(),
                                                    endog_colname,
                                                    gbkey,
                                                    ds_column, 
                                                    freq, 
                                                    fcst_length, 
                                                    run_type, 
                                                    holdout_length, 
                                                    model_list,
                                                    fit_execution_method,
                                            )

result.head()

In [ ]:
# m_list = model_list.remove('prophet')
for k, df in result.groupby('ts_id'):
    df.iloc[-50:].plot(y=[x+'_forecast' for x in model_list])


In [ ]:
for item in fit_result_list:
    for model in model_list:
        if item[model+'_err'] is not None:
            print(model + ': ' + item[model+'_err'])